In [185]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm

import torchvision
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.models as models
import warnings
warnings.filterwarnings('ignore')
import os
# from google.colab import drive
# drive.mount('/content/drive')

In [186]:
# os.chdir('/content/drive/MyDrive/딥러닝홀로서기')

In [187]:
# !pip install efficientnet_pytorch

In [188]:
from efficientnet_pytorch import EfficientNet
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 디바이스 설정
print(device)

cuda:0


In [189]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [190]:
class FashionMNIST(D.Dataset):
    def __init__(self,data,label,transform=None):
        self.data = data
        self.label = label
        self.transform = transform
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        image = self.data[idx]
        image = image.reshape(-1, 28, 28).astype('float32')
        label = self.label[idx]
        if self.transform:
            image = self.transform(image)
            image = image.transpose(0,1)
        return image,label

In [191]:
class PretrainModel(nn.Module):
    def __init__(self,pretrained):
        super().__init__()
        self.conv2d = nn.Conv2d(1, 3, 3, stride=1)
        self.pretrained = pretrained
#         self.pretrained_fc = nn.Linear(1000,512)
#         self.dropout = nn.Dropout(p = 0.3)
#         self.activation = nn.SiLU()
        self.FC = nn.Linear(1000,10)
    
    def forward(self,x):
        x = self.conv2d(x)
        x = self.pretrained(x)
#         x = self.pretrained_fc(x)
#         x = self.dropout(x)
#         x = self.activation(x)
        x = self.FC(x)
        return x


In [192]:
# class FashionCNN(nn.Module):
    
#     def __init__(self):
#         super(FashionCNN, self).__init__()
        
#         self.layer1 = nn.Sequential(
#             nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2)
#         )
        
#         self.layer2 = nn.Sequential(
#             nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(2)
#         )
        
#         self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
#         self.drop = nn.Dropout2d(0.25)
#         self.fc2 = nn.Linear(in_features=600, out_features=120)
#         self.fc3 = nn.Linear(in_features=120, out_features=10)
        
#     def forward(self, x):
#         out = self.layer1(x)
#         out = self.layer2(out)
#         out = out.view(out.size(0), -1)
#         out = self.fc1(out)
#         out = self.drop(out)
#         out = self.fc2(out)
#         out = self.fc3(out)
        
#         return out

In [193]:
transformer = T.Compose([
    T.ToTensor(),
    T.Normalize((0.1307,), (0.3081,)),
    T.RandomHorizontalFlip(),
    T.RandomAffine(30)
])

In [194]:
pixel = train.to_numpy()[:,2:]
labels = train.to_numpy()[:,1]
BATCH_SIZE = 64
# mobilenet_v3_large = models.mobilenet_v3_large(pretrained=True)
pretrained = EfficientNet.from_pretrained('efficientnet-b7')
model = PretrainModel(pretrained)
# model = nn.DataParallel(FashionCNN())
model = nn.DataParallel(model)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(),lr = 1e-3)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=2, eta_min=0.001)
criterion = torch.nn.CrossEntropyLoss()

Loaded pretrained weights for efficientnet-b7


In [195]:
dataset = FashionMNIST(pixel,labels,transformer)
train_dataset, val_dataset = D.random_split(dataset, [len(dataset) - int(len(dataset) * 0.1), int(len(dataset) * 0.1)])
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size = BATCH_SIZE,shuffle=True, drop_last = True)
val_dataloader = torch.utils.data.DataLoader(val_dataset,batch_size = BATCH_SIZE, shuffle=True,drop_last = True)

In [ ]:
total_step = len(train_dataloader)
batch_val_acc = 0
EPOCH = 50

for epoch in range(EPOCH):
    train_acc_list = []
    running_loss = 0.0
    
    model.train()

    for i,(images,labels) in tqdm(enumerate(train_dataloader)):
        images = images.type(torch.FloatTensor).to(device)
        labels = labels.type(torch.LongTensor).to(device)

        optimizer.zero_grad()
        probs = model(images)

        loss = criterion(probs,labels)
        loss.backward()
        optimizer.step()


        running_loss += loss.item()

        probs = probs.cpu().detach().numpy()
        labels = labels.cpu().detach().numpy()
        preds = np.argmax(probs,1)
        # print(probs)
        # print((preds == labels))
        batch_acc =  (preds == labels).sum() / BATCH_SIZE * 100
        train_acc_list.append(batch_acc)
    
    train_acc = np.mean(train_acc_list)
    print(f'Epoch [{epoch+1}/{EPOCH}], Step [{i+1}/{total_step}], Loss: {running_loss/total_step}, Acc {train_acc}')

    model.eval()
    valid_acc_list = []
    with torch.no_grad():
        correct = 0
        total = 0

        for images, labels in val_dataloader:
            images = images.type(torch.FloatTensor).to(device)
            labels = labels.type(torch.LongTensor).to(device)

            probs = model(images)
            valid_loss = criterion(probs, labels)

            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = np.argmax(probs,1)
            batch_acc =  (preds == labels).sum() / BATCH_SIZE * 100
            valid_acc_list.append(batch_acc)
            
        val_acc = np.mean(valid_acc_list)
        print(f'Validation acc: {val_acc}')

    lr_scheduler.step()


843it [02:35,  5.44it/s]


Epoch [1/50], Step [843/843], Loss: 1.0709359354537293, Acc 61.72709074733096


1it [00:00,  5.33it/s]

Validation acc: 55.42674731182796


843it [02:33,  5.48it/s]


...

843it [02:34,  5.46it/s]


Epoch [4/50], Step [843/843], Loss: 0.7885638726422229, Acc 72.31983985765125


0it [00:00, ?it/s]

Validation acc: 71.99260752688173


209it [00:38,  5.39it/s]

In [85]:
pixel = test.to_numpy()[:,1:]
labels = submission.to_numpy()[:,1] # 의미 X
test_dataset = FashionMNIST(pixel,labels)
test_dataloader = torch.utils.data.DataLoader(test_dataset,shuffle=False, drop_last = False)

In [86]:
model.eval()
predict_list = []
with torch.no_grad():
    for images,labels in tqdm(test_dataloader):
        images = images.type(torch.FloatTensor).to(device)
        labels = labels.type(torch.LongTensor).to(device)

        probs = model(images)
        probs  = probs.cpu().detach().numpy()
        preds = np.argmax(probs,1)
        predict_list.append(preds[0].astype(np.int))

100%|████████████████████████████████████████████████████████████████████████████| 10000/10000 [07:23<00:00, 22.57it/s]


In [87]:
submission['label'] = predict_list

In [88]:
submission.to_csv('eff-b7_transform.csv',index=False)